In [ ]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import join, isfile, expandvars

# some parameters
binning = 250
smoothing = 5
exposure = 100

# info tables
infotable = f'{expandvars("$HOME")}/E4/irf_random/crab/dl3_random_irf/merged_simulator_data.dat'
infodata = pd.read_csv(infotable, sep=' ', header=0).sort_values(by=['seed'])
infodata.head()

In [ ]:
import matplotlib.pyplot as plt
import astropy.units as u
from astropy import wcs
from astroai.tools.utils import *

configuration = f'{expandvars("$HOME")}/astroAI/astroai/conf/regressor.yml'
conf = load_yaml_conf(configuration)
conf = conf['preprocess']
trange = [conf['time_start'], conf['time_stop']]
conf

In [ ]:
f = f'{expandvars("$HOME")}/E4/irf_random/crab/sim/crab_03532.fits'
m = f'{expandvars("$HOME")}/E4/irf_random/crab/map_test_regressor/noisy/crab_03532_map.fits'
seed = int(''.join(filter(str.isdigit, basename(f))))
seed

In [ ]:
row = infodata[infodata['seed']==seed]
row

# Check DL3 

In [ ]:
from astropy.io import fits

# check from DL3
with fits.open(f) as h:
    hdr = h[1].header
    data = h[1].data
hdr['RA_PNT'], hdr['DEC_PNT'], hdr

# Set WCS from DL4

In [ ]:
# check from DL4
with fits.open(m) as h:
    hdr = h[0].header
    data = h[0].data
    wcs_skymap = WCS(h[0].header)
wcs_skymap, hdr, data.shape

In [ ]:
# LABEL: convert source to pixel coordinates
xdl4, ydl4 = wcs_fits.world_to_pixel(SkyCoord(row['source_ra'].values[0], row['source_dec'].values[0],
                                        unit='deg', frame='icrs'))
xdl4, ydl4

# Set WCS with center = pointing

In [ ]:
# set wcs
wcs_fits = set_wcs(point_ra=row['point_ra'].values[0], point_dec=row['point_dec'].values[0],
                   point_ref=binning/2+0.5, pixelsize=(2*row['fov'].values[0])/binning)
wcs_fits

In [ ]:
# LABEL: convert source to pixel coordinates
xfits, yfits = wcs_fits.world_to_pixel(SkyCoord(row['source_ra'].values[0], row['source_dec'].values[0],
                                        unit='deg', frame='icrs'))
xfits, yfits

# Set WCS with center = (0,0) of matrix

In [ ]:
# set wcs
wcs_npy = set_wcs(point_ra=row['point_ra'].values[0], point_dec=row['point_dec'].values[0],
                  point_ref=0, pixelsize=(2*row['fov'].values[0])/binning)
# LABEL: convert source to pixel coordinates
xnpy, ynpy = wcs_npy.world_to_pixel(SkyCoord(row['source_ra'].values[0], row['source_dec'].values[0],
                                       unit='deg', frame='icrs'))
xnpy, ynpy

In [ ]:
sky_fits = wcs_fits.pixel_to_world(xfits, yfits)
sky_npy = wcs_npy.pixel_to_world(xnpy, ynpy)
print('TRUE coordinates sim:    ', row['source_ra'].values[0], row['source_dec'].values[0])
print('FITS WCS on NPY heatmap: ', sky_fits.ra.deg, sky_fits.dec.deg)
print('NPY WCS on NPY heatmap:  ', sky_npy.ra.deg, sky_npy.dec.deg)

# Verify WCS in plots

In [ ]:
dl3 = Table.read(f, hdu=1).to_pandas()
heatmap, xe, ye = extract_heatmap_from_table(dl3, smoothing, binning, filter=True, trange=(0, exposure))
heatmap.shape, heatmap.max(), heatmap.min()

In [ ]:
src_fits = (xfits, yfits)
src_npy = (xnpy, ynpy)

plot_heatmap_wcs(heatmap, title=f'raw heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=None, src=None, pnt=None)

plot_heatmap_wcs(heatmap, title=f'raw FITS heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_fits, src=None, pnt=None)

plot_heatmap_wcs(heatmap, title=f'raw NPY heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_npy, src=None, pnt=None)

# Stretch and normalise heatmap

In [ ]:
stretch_heatmap = stretch_smooth(heatmap, smoothing)

In [ ]:
plot_heatmap_wcs(stretch_heatmap, title=f'stretch heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=None, src=None, pnt=None)

plot_heatmap_wcs(stretch_heatmap, title=f'stretch FITS heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_fits, src=None, pnt=None)

plot_heatmap_wcs(stretch_heatmap, title=f'stretch NPY heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_npy, src=None, pnt=None)

# Normalise heatmap

In [ ]:
normed_heatmap = normalise_heatmap(heatmap)

In [ ]:
plot_heatmap_wcs(normed_heatmap, title=f'stretch heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=None, src=None, pnt=None)

plot_heatmap_wcs(normed_heatmap, title=f'stretch FITS heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_fits, src=None, pnt=None)

plot_heatmap_wcs(normed_heatmap, title=f'stretch NPY heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_npy, src=None, pnt=None)

# Normalise DS

In [ ]:
norm_value = 0.5
ds_normed_heatmap = normalise_dataset(heatmap, max_value=norm_value)

In [ ]:
plot_heatmap_wcs(ds_normed_heatmap, title=f'stretch heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=None, src=None, pnt=None)

plot_heatmap_wcs(ds_normed_heatmap, title=f'stretch FITS heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_fits, src=None, pnt=None)

plot_heatmap_wcs(ds_normed_heatmap, title=f'stretch NPY heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_npy, src=None, pnt=None)

# Stretch and normalise DS

In [ ]:
ds_stretch_heatmap = stretch_min_max(heatmap, vmax=norm_value)

In [ ]:
plot_heatmap_wcs(ds_stretch_heatmap, title=f'stretch heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=None, src=None, pnt=None)

plot_heatmap_wcs(ds_stretch_heatmap, title=f'stretch FITS heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_fits, src=None, pnt=None)

plot_heatmap_wcs(ds_stretch_heatmap, title=f'stretch NPY heatmap {seed}', xlabel='x', ylabel='y',
                 show=True, wcs=wcs_npy, src=None, pnt=None)